In [ ]:
import sys 
sys.path.append(r'G:\My Drive\src\Trace')
import pims
import tracetools
from matplotlib import pyplot as plt
%matplotlib inline
import scipy
import os.path
import imp
import glob
tracetools = imp.reload(tracetools)
import pandas

import itertools


In [ ]:
basename   =            'yRS102_20191014_EX312_2'
directory  =            "D:\\ex312\\20191014\\yRS102_20191014_EX312_2\\"
movie_url  =            directory + basename + "_BS_BC_SLICED_MAX.tif"
loc_url    =            directory + basename + "_discover.loc"


cell_paths =  glob.glob(directory + basename + "_*.zip")

print(basename,directory,movie_url,loc_url,sep="\n")

In [ ]:
##Creating the loc file  for a  tif
with pims.open(movie_url) as tiff:
    loc = scipy.array([puncta for puncta in tracetools.getAllPunctaFromStack(tiff,psfPx = 1.5, window_radius = 40)])
scipy.savetxt(os.path.join(directory,loc_url) ,loc,delimiter='\t',fmt=['%d','%d','%f','%d'])

In [ ]:
loc = scipy.loadtxt(os.path.join(directory,loc_url),delimiter='\t',converters={0:int,1:int,2:float,3:int})

In [ ]:
cell_paths =  glob.glob(directory + basename + "_*.zip")

In [ ]:
import read_roi
tracks = []
for path in cell_paths:
    print(path)
    roi_path     = os.path.join(directory,path)
    cell_number  = os.path.basename(roi_path).split('.')[0].split('_')[-1]
    fname        = os.path.basename(movie_url).split('.')[0]+"_"+str(cell_number)+'.trk'
    track_path   = os.path.join(os.path.dirname(roi_path),fname     )
    rois         = read_roi.read_roi_zip(roi_path)
    rough_track  = tracetools.trackFromCellKeyFrames(rois,loc,0)
    tracks.append([rough_track,track_path])
tracks, paths = zip(*tracks)

In [ ]:
sloc = scipy.array(sorted(tracetools.mergeTracks(tracks), key = lambda x:x[3]))
with pims.open(movie_url) as tiff:
    nloc = scipy.array(list(tracetools.reAnalyzePuncta(tiff, sloc)))
tracks = scipy.array([nloc[nloc[:,4]==i] for i in sorted(list(set(nloc[:,4])))])

In [ ]:
print(scipy.array(cell_paths)[[any(scipy.isnan(track[:,2])) for track in tracks]])
print(scipy.array([len(track) for track in tracks]))


In [ ]:
for rough_track, path in zip(tracks,paths):
    print(path)
    #plotting  the track
    plt.figure(figsize=(15,5))
    #plt.plot(rough_track[:,3], scipy.signal.medfilt(rough_track[:,2],11))
    plt.plot(rough_track[:,3], rough_track[:,2])
    plt.ylim([0,scipy.amax(tracks.flatten())])
    plt.figure(figsize=(1,1))
    plt.plot(rough_track[:,0],rough_track[:,1])

fig             = plt.figure(figsize=(10,5),dpi=300,)
main_ax         = fig.add_axes([1,1,1,1])
main_ax.matshow([scipy.signal.medfilt(track[:,2],11) for track in tracks],aspect='auto')
#main_ax.matshow(scipy.array([track[:,2] for track in tracks]),aspect='auto')

In [ ]:
for track,path in zip(tracks,paths):
    scipy.savetxt(path, track, delimiter='\t',fmt=['%d','%d','%f','%d','%d'])

In [ ]:
other_url  =  movie_url.replace("BS_BC_","BS_")
sloc = scipy.array(sorted(tracetools.mergeTracks(tracks), key = lambda x:x[3]))
with pims.open(other_url) as tiff:
    nloc = scipy.array(list(tracetools.reAnalyzePuncta(tiff, sloc)))
tracks = scipy.array([nloc[nloc[:,4]==i] for i in sorted(list(set(nloc[:,4])))])
for track,path in zip(tracks,[path.replace("BS_BC_","") for path in paths]):
    scipy.savetxt(path, track, delimiter='\t',fmt=['%d','%d','%f','%d','%d'])

In [ ]:
for rough_track in tracks:
    #plotting  the track
    plt.figure(figsize=(15,5))
    #plt.plot(rough_track[:,3], scipy.signal.medfilt(rough_track[:,2],11))
    plt.plot(rough_track[:,3], rough_track[:,2])
    plt.ylim([0,scipy.amax(tracks)])
    plt.figure(figsize=(1,1))
    plt.plot(rough_track[:,0],rough_track[:,1])
fig             = plt.figure(figsize=(10,5),dpi=300,)
main_ax         = fig.add_axes([1,1,1,1])
main_ax.matshow([scipy.signal.medfilt(track[:,2],11) for track in tracks],aspect='auto')
#main_ax.matshow(scipy.array([track[:,2] for track in tracks]),aspect='auto')

In [ ]:
other_url  =  movie_url.replace("MAX","SUM")
sloc = scipy.array(sorted(tracetools.mergeTracks(tracks), key = lambda x:x[3]))
with pims.open(other_url) as tiff:
    nloc = scipy.array(list(tracetools.reAnalyzePuncta(tiff, sloc)))
tracks = scipy.array([nloc[nloc[:,4]==i] for i in sorted(list(set(nloc[:,4])))])
for track,path in zip(tracks,[path.replace("MAX","SUM") for path in paths]):
    scipy.savetxt(path, track, delimiter='\t',fmt=['%d','%d','%f','%d','%d'])

In [ ]:
for rough_track in tracks:
    #plotting  the track
    plt.figure(figsize=(15,5))
    #plt.plot(rough_track[:,3], scipy.signal.medfilt(rough_track[:,2],11))
    plt.plot(rough_track[:,3], rough_track[:,2])
    plt.ylim([0,scipy.amax(tracks)])
    plt.figure(figsize=(1,1))
    plt.plot(rough_track[:,0],rough_track[:,1])
fig             = plt.figure(figsize=(10,5),dpi=300,)
main_ax         = fig.add_axes([1,1,1,1])
main_ax.matshow([scipy.signal.medfilt(track[:,2],11) for track in tracks],aspect='auto')
#main_ax.matshow(scipy.array([track[:,2] for track in tracks]),aspect='auto')

In [ ]:
tiff.close()